# Fitting Low Flow Distributions to Estimate 7Q10

In [ ]:
!pip install dataretrieval
!pip install lmoments3

In [ ]:
from google.colab import drive
import numpy as np
import scipy.stats as ss
import pandas as pd
import matplotlib.pyplot as plt
import dataretrieval.nwis as nwis

# allow access to google drive
drive.mount('/content/drive')

!cp "drive/MyDrive/Colab Notebooks/CE6280/CodingExamples/utils.py" .
from utils import *

In [ ]:
flow_df = nwis.get_record(sites='05412500', service='dv', parameterCd='00060', start='1932-10-01', end='2024-09-30') # Turkey River at Garber, IA

# find water year of each data point
flow_df['Year'] = flow_df.index.year
flow_df['Month'] = flow_df.index.month
flow_df['WY'] = flow_df['Year']
flow_df['WY'][np.where(flow_df['Month']>=10)[0]] = flow_df['WY'][np.where(flow_df['Month']>=10)[0]] + 1

# find minimum 7-day flows each year
years = np.arange(np.min(flow_df['WY']), np.max(flow_df['WY'])+1, 1)
min7dayQ = np.zeros(len(years))
for i,year in enumerate(years):
    yearlyData = np.array(flow_df['00060_Mean'])[np.where(flow_df['WY']==year)[0],]
    sevenDayQ = np.zeros(len(yearlyData)-7+1)
    for j in range(len(yearlyData)-7+1):
        sevenDayQ[j] = np.mean(yearlyData[j:(j+7)])

    min7dayQ[i] = np.min(sevenDayQ)

plt.plot(min7dayQ)

Theoretically, the distribution of 7-day annual minima should have a Weibull distribution. Let's write a class to fit a Weibull distribution using MLE, MOM or L-moments.

In [ ]:
from math import gamma as GammaFN
from scipy.optimize import brentq as root
from scipy.optimize import fsolve
from lmoments3 import distr

class Weibull(Distribution):
  def __init__(self):
    super().__init__()
    self.alpha = None
    self.kappa = None
    self.xi = None

  def fit(self, data, method, npars, initialize=True, initialMethod='MOM'):
    assert method == 'MLE' or method == 'MOM' or method == "Lmom","method must = 'MLE','MOM', or 'Lmom'"
    assert npars == 2 or npars == 3,"npars must = 2 or 3"

    self.findMoments(data)
    self.findLmoments(data)
    if method == 'MLE':
      if initialize == False:
        if npars == 2:
          kappa, xi, alpha = ss.weibull_min.fit(data,floc=0)
        elif npars == 3:
          kappa, xi, alpha = ss.weibull_min.fit(data)
      else:
        if npars == 2:
          if initialMethod == 'MOM':
            self.fit(data, 'MOM', 2)
          elif initialMethod == 'Lmom':
            self.fit(data, 'Lmom', 2)
          kappa, xi, alpha = ss.weibull_min.fit(data, self.kappa, floc=0)
        elif npars == 3:
          if initialMethod == 'MOM':
            self.fit(data, 'MOM', 3)
          elif initialMethod == 'Lmom':
            self.fit(data, 'Lmom', 3)
          self.fit(data, 'Lmom', 3)
          kappa, xi, alpha = ss.weibull_min.fit(data, self.kappa)
    elif method == 'MOM':
      if npars == 2:
        self.kappa = root(lambda x: self.xbar**2 * (GammaFN(1+2/x)/GammaFN(1+1/x)**2 -1) - self.var, 0.02, 10)
        self.alpha = self.xbar / GammaFN(1+1/self.kappa)
        self.xi = 0
      elif npars == 3:
        def equations(p):
          kappa, xi, alpha = p
          mu = alpha*GammaFN(1+1/kappa) + xi
          sigma = np.sqrt(alpha**2*(GammaFN(1+2/kappa)-(GammaFN(1+1/kappa))**2))
          gamma = (GammaFN(1+3/kappa)*alpha**3 -3*mu*sigma**2 - mu**3)/(sigma**3)
          return (mu-self.xbar, sigma**2-self.var, gamma-self.skew)
        # initialize parameter estimates at Weibull 2 MOM parameter
        kappa = root(lambda x: self.xbar**2 * (GammaFN(1+2/x)/GammaFN(1+1/x)**2 -1) -
                     self.var, 0.02, 10)
        alpha = self.xbar / GammaFN(1+1/kappa)
        xi = 0
        self.kappa, self.xi, self.alpha = fsolve(equations,(kappa,xi,alpha))
    elif method == 'Lmom':
      if npars == 2:
        self.xi = 0
        self.kappa = root(lambda x: self.L2/self.L1 - 1 + 1/(2**(1/x)),0.02,10)
        self.alpha = self.L1 / GammaFN(1+1/self.kappa)
      elif npars == 3:
        weibull_params = distr.wei.lmom_fit(data)
        self.kappa = weibull_params["c"]
        self.xi = weibull_params["loc"]
        self.alpha = weibull_params["scale"]

  def findReturnPd(self, T):
    q_T = ss.weibull_min.ppf(1-1/T, self.kappa, self.xi, self.alpha)
    return q_T

  def plotHistPDF(self, data, min, max, title):
    x = np.arange(min, max,(max-min)/100)
    f_x = ss.weibull_min.pdf(x, self.kappa, self.xi, self.alpha)

    plt.hist(data, density=True)
    plt.plot(x,f_x)
    plt.xlim([min, max])
    plt.title(title)
    plt.xlabel('Flow')
    plt.ylabel('Probability Density')
    plt.show()

Now fit the Weibull distribution with 2 or 3 parameters using each method.

In [ ]:
methods = ["MOM", "MLE", "Lmom"]
npars = [2, 3]

for method in methods:
  for npar in npars:
    distfit = Weibull()
    distfit.fit(min7dayQ, method, npar)
    sevenQ10 = distfit.findReturnPd(1/(1-0.1))
    distfit.plotHistPDF(min7dayQ, 0, 1000, "Weibull" + str(npar) + " " + str(method) + " Fit")
    print("Weibull%d %s kappa: %0.2f" % (npar, method, distfit.kappa))
    print("Weibull%d %s xi: %0.2f" % (npar, method, distfit.xi))
    print("Weibull%d %s alpha: %0.2f" % (npar, method, distfit.alpha))
    print("Weibull%d %s 7Q10: %0.0f cms" % (npar, method, sevenQ10))
    print("\n")

We could extend this to capture non-stationarity in the moments/parameters of the Weibull distribution since we saw in HW5 that there is a statistically significant trend, even after controlling for autocorrelation.